# Practice Session 04: Networks from text

Author: <font color="blue">Rubén Vera</font>

E-mail: <font color="blue">ruben.vera01@estudiant.upf.edu</font>

Date: <font color="blue">22/10/2021</font>

# 1. Create the directed mention network

In [46]:
import io
import json
import gzip
import csv
import re

In [47]:
# Leave this code as-is

# Input file
COMPRESSED_INPUT_FILENAME = "CovidLockdownCatalonia.json.gz"

# These are the output files, leave as-is
OUTPUT_ALL_EDGES_FILENAME = "CovidLockdownCatalonia.csv"
OUTPUT_FILTERED_EDGES_FILENAME = "CovidLockdownCatalonia-min-weight-filtered.csv"
OUTPUT_CO_MENTIONS_FILENAME = "CovidLockdownCatalonia-co-mentions.csv"

## 1.1. Extract mentions

In [48]:
# Leave this code as-is

def extract_mentions(text):
    return re.findall("@([a-zA-Z0-9_]{5,20})", text)

print(extract_mentions("RT @Jordi: check this post by @Xavier"))

['Jordi', 'Xavier']


## 1.2. Count mentions

In [49]:
mentions_counter = {}
with gzip.open(COMPRESSED_INPUT_FILENAME, "rt", encoding="utf-8") as input_file:
    for line in input_file:
        tweet = json.loads(line)
        author = tweet["user"]["screen_name"]
        message = tweet["full_text"]
        mentions = extract_mentions(message)
        for mention in mentions:
            key = (author, mention)
            if key in mentions_counter:
                mentions_counter[key] += 1
            else:
                mentions_counter[key] = 1


In [50]:
print(mentions_counter['joanmariapique', 'catalangov'])

9


In [51]:
# Leave this code as-is

with io.open(OUTPUT_ALL_EDGES_FILENAME, "w") as output_file:
    writer = csv.writer(output_file, delimiter='\t', quotechar='"')
    writer.writerow(["Source", "Target", "Weight"])
    for key in mentions_counter:
        author = key[0]
        mention = key[1]
        weight = mentions_counter[key]
        writer.writerow([author, mention, weight])

In [52]:
with io.open(OUTPUT_FILTERED_EDGES_FILENAME, "w") as output_file:
    writer = csv.writer(output_file, delimiter='\t', quotechar='"', lineterminator='\n')
    writer.writerow(["Source", "Target", "Weight"])
    for key in mentions_counter:
        author = key[0]
        mention = key[1]
        weight = mentions_counter[key]
        if(weight >= 2):
            writer.writerow([author, mention, weight])

# 2. Visualize the directed mention network

Open the **filtered** edge file in Cytoscape, by importing its CSV file. You may have to set the delimiter to "Tab" in the advanced options, when importing.

The file is large so if you want to see all details while zooming out you may have to set ``View > Always show Graphic Details``. Note this makes the program run slower.

Style the network:

* Run "Tools > Analyze Network ..." (as a directed graph)
* Style nodes by setting their size proportional to their in-degree
* Style edges by setting their width and color (darker=more) using the *weight* attribute.

*Tip*: to count nodes in Cytoscape, hold shift while clicking and select the nodes. In the lower-right corner you should see a count of nodes and edges.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

 ![Mentions graph](mentions.png)

En este grafo solo podemos encontrar una componente conexa, de 699 nodos, los cuales son 699 perfiles de twitter que han mencionado a otros perfiles o que han sido mencionados por otros. Se puede pareciar que hay ciertos nodos con mayor grado que se encuentran rodeados de muchos otros nodos probablemente porque esos nodos son personas mas populares las cuales se les suele mencionar con frecuencia y los nodos que los rodean son personas mas anonimas que no tienen mas interacciones que el hecho de mencionar al nodo central de su grupo.

![Mentions graph largest connected component](mentions-largest-cc.png)

 ![Mentions in-degree distribution](mentions-in-degree-distribution.png) ![Mentions out-degree distribution](mentions-out-degree-distribution.png)

Este grafo es un grafo dirigido, donde los nodos con mas centralidad son como se ha mencionado en la respuesta anterior, aquellos nodos mas populares que tienden a tener muchas mas interacciones con otros usuarios por estar relacionado con el tema de los tweets extraidos. Un dato curioso es que en cuanto al grado de entrada, 686/699 nodos estan entre grado 0 y grado 5, mientras que solo 13 nodos superan ese grado. Lo curioso es que en el grado de salida nos encontramos con mas o menos los mismos valores, 685/699 nodos estan entre grado 0 y grado 5. Por lo que los usuarios seleccionados tienden a tener el mismo grado de salida que de entrada y suele estar entre el 0 y el 5.

# 3. Create the undirected co-mention network

In [53]:
co_mentions_counter = {}
with gzip.open(COMPRESSED_INPUT_FILENAME, "rt", encoding="utf-8") as input_file:
    for line in input_file:
        tweet = json.loads(line)
        author = tweet["user"]["screen_name"]
        message = tweet["full_text"]
        mentions = extract_mentions(message)
        for mention1 in mentions:
            for mention2 in mentions:
                if mention1 < mention2:
                    key = (mention1, mention2)
                    if key in co_mentions_counter:
                        co_mentions_counter[key] += 1
                    else:
                        co_mentions_counter[key] = 1

In [54]:
print(co_mentions_counter['agriculturacat','uniopagesos'])

8


In [55]:
with io.open(OUTPUT_CO_MENTIONS_FILENAME, "w") as output_file:
    writer = csv.writer(output_file, delimiter='\t', quotechar='"')
    writer.writerow(["Source", "Target", "Weight"])
    for key in co_mentions_counter:
        mention1 = key[0]
        mention2 = key[1]
        weight = co_mentions_counter[key]
        writer.writerow([mention1, mention2, weight])

![Co-mentions graph](co-mentions.png)

Los nodos mas centricos son aquellos con mayor grado y por consecuencia son aquellos que mas relación pueden tener en el tema que se trata en estos tweets, es decir son personas importantes en el ambito gubernamental o en el ambito de la salud. En cuanto al numero de nodos, al final siendo twitter una red social tan libre parece comprensible que pueda haber tantos nodos por el hecho de que cualquiera puede mencionar a otra persona. La componente conexa con mayor numero de nodos es aquella donde se encuentra la mayoria de la parte gubernamental y de la salud del país al igual que se menciona con los nodos de mayor grado. Las otras componentes son pequeños grupos que entre ellos tienen alta comunicación y a causa de ello se mencionan mutuamente sobre este tema ya que lo hacen con cualquier otro.

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, text, and figures were produced by myself.</font>
